In [2]:
import pandas
import pickle
import zipfile
import datetime
from tqdm import tqdm
from catalog_class import Catalog

In [ ]:
catalogues_file = 'data\\catalogues.pickle'
catalogues = Catalog(pickle_file=catalogues_file)

14it [04:48, 64.74s/it]

In [ ]:
from project_tools.os_tools import generator_files_in_dir

filtered_dir = r"E:\Google Drive nikolaev.ivan.rf\vacancies\filtered_id_catalogues_date"

dfs = []
for file_zip in tqdm(generator_files_in_dir(filtered_dir, extension='.zip')):
    with zipfile.ZipFile(file_zip) as opened_zip:
        for pickle_file_in_zip in opened_zip.namelist():
            with opened_zip.open(pickle_file_in_zip, 'r') as mypicklefile:
                df = pickle.load(mypicklefile)
                dfs.append(df)

In [ ]:
df = pandas.concat(dfs, ignore_index=True)
df.head(3)
print(len(df))

In [ ]:
df['date_published_normal'] = df['date_published'].apply(datetime.datetime.fromtimestamp)
df['catalogues_len'] = df['catalogues'].apply(len)
df.head()

In [ ]:
# распределение вакансий по количеству catalogues у одной вакансии
df['catalogues_len'].value_counts()

In [ ]:
df_catalogues_len_1 = df[df['catalogues_len'] == 1]
df_catalogues_len_1['catalogues_1'] = df_catalogues_len_1['catalogues'].apply(lambda x: x[0])
df_catalogues_len_1.head()

In [ ]:
len(df_catalogues_len_1)

In [ ]:
from matplotlib.pyplot import xticks
dfboth = df_catalogues_len_1.groupby([df_catalogues_len_1['date_published_normal'].dt.strftime('%y-%m'),'catalogues_1']).count()['catalogues_len'].unstack()
dfboth.columns = [str(col)+" "+catalogues.get_name_catalog_by_key(col) for col in dfboth.columns]

dfboth.plot(figsize=(30,15), title="Test", xticks=range(len(dfboth)), grid=True, rot=90)

In [ ]:
import matplotlib.pylab as plt
import matplotlib.dates as mdates


for key, title in catalogues.generator_keys_catalogues():
  fig, ax = plt.subplots(figsize=(15,7))

  df_catalog = df_catalogues_len_1.loc[df_catalogues_len_1['catalogues_1'] == key]
  df_ = df_catalog.groupby(df_catalog['date_published_normal'].dt.strftime('%y-%m')).count()
  ax = df_['date_published_normal'].plot(title = title)
  ax.set_xticks(range(len(dfboth)));
  ax.set_xticklabels([item for item in dfboth.index.tolist()], rotation=90);